In [2]:
import pandas as pd
import numpy as np

# Define sheet name
FILE_NAME = '../Sources/Data.xlsx'

# Get all sheet name in Excel file
sheet_name_list = pd.ExcelFile(FILE_NAME).sheet_names

# Show all sheet name and index number
for index, sheet_name in enumerate(sheet_name_list):
    print(f"{index} : {sheet_name}")

# Read data base on selected SELECT_SHEET_NAME
SELECT_SHEET_NAME = sheet_name_list[6]

# add dataframe for selected sheet
df = pd.read_excel(FILE_NAME, sheet_name= SELECT_SHEET_NAME, skiprows = 5)

0 : Index
1 : Vol. of Plastic card usage
2 : Value. of Plastic card usage
3 : Payment Cards on ATM & POS 
4 : Mobile,internet banking
5 : e-Money
6 : promptpay


In [3]:

### Data Cleansing ###

# Change column name and drop un-related column
df.rename(columns = {"Unnamed: 1" : "Attribute"}, inplace = True)
df.dropna(subset = ["Attribute"], inplace = True)
df.drop(columns = ["Unnamed: 0"], inplace = True)
df.replace("n.a.", np.nan, inplace = True)

# Cleansing Column name (trim and remove leeter after year)
col_list_clean = []
col_list_trim = [col_name.strip() for col_name in df.columns]
for col_name in col_list_trim:
    if len(col_name.split(" ")) <= 2:
        col_list_clean.append(col_name)
    else:
        cleaned_col_name =  " ".join(col_name.split(" ")[:-1])
        col_list_clean.append(cleaned_col_name)
        
# Overwrite existing column with new columns clean column    
df.columns = col_list_clean

# replace value
df['Attribute'] = df['Attribute'].replace(['Volume of transactions (Thousand Transactions)', 'Value of transactions (Billions of Baht)'], ['volume_k', 'value_b'])

In [4]:
### Data Shaping ###

# Unpivot MonthYear Column
value_col_list = [col_name for col_name in df.columns if col_name != "Attribute"]
df = pd.melt(df, id_vars = ["Attribute"], value_vars = value_col_list)

In [5]:
# rename column
df.rename(columns = {"Attribute" : "MeasureName" ,"variable" : "MonthYear" , "value" : "M_Value"}, inplace = True)

# addcolumn
df['TransType'] = 'PromptPay'
df['ChannelType'] = 'PromptPay'

In [6]:
# save output to csv
df.to_csv(f'../Clean/PromptPay_Clean.csv', index = False)
